In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np

import torch
from torch import nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from src.network_analysis import count_parameters
from src.networks.skip_nets.dense_skip_net import DenseSkipNet
from src.networks.core.torch_net import TorchNet
from src.networks.core.layer_connections import LayerConnections
from src.networks.skip_nets.additive_skip_net import AdditiveSkipNet

def create_layer(i, last, in_features, out_features):
    l = nn.Linear(in_features, out_features)
    l.weight.data[:] = 1.0
    l.bias.data[:] = 0.0
    return nn.Sequential(
        l,
        nn.CELU(),
        # nn.Linear(out_features, out_features),
        # nn.CELU(),
        # nn.Linear(out_features, out_features),
        # nn.CELU(),
    )

num_features=41
net = AdditiveSkipNet.from_layer_provider(
    layer_provider=create_layer,
    num_layers=4,
    num_features=num_features,
    layer_connections='full',
    initial_direct_connection_weight=1.0,
    initial_skip_connection_weight=0.01,
    return_dense=False,
    weights_trainable=False,
)
# net = DenseSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=16,
#     num_features=num_features,
#     layer_connections='full'
# )
print(count_parameters(net))
print(net)

net(torch.ones(128, 128, 16, num_features)).shape


7503
AdditiveSkipNet(
  (layers): NetList(
    (0-3): 4 x TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=41, out_features=41, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
  )
)


torch.Size([128, 128, 16, 41])

In [34]:
2 * 9 + 0.1 * 2 * (4.2 + 2 + 1)

19.44